In [ ]:
!pip install "docling[all]" pymupdf

In [ ]:
import fitz  # PyMuPDF
import os
import time
from pathlib import Path
from docling.document_converter import DocumentConverter, PdfFormatOption
from docling.datamodel.pipeline_options import PdfPipelineOptions
from docling.datamodel.base_models import InputFormat

# --- PARÁMETROS DE CONFIGURACIÓN ---
# Asegúrate de que este archivo exista en tu entorno de Colab/local
PDF_FILE = Path("/content/PDF-GenAI-Challenge.pdf") 
OUTPUT_MD = Path("/content/output_completo_robusto.md")
CHUNK_SIZE = 5 
# ------------------------------------

def process_pdf_with_fallback(pdf_path: Path, output_md: Path, chunk_size: int):
    """
    Convierte un PDF completo a Markdown, procesándolo en chunks.
    Implementa una lógica de fallback con PyMuPDF si la conversión principal con Docling falla.
    """
    if not pdf_path.exists():
        print(f"❌ Error: El archivo '{pdf_path}' no existe. Por favor, verifica la ruta.")
        return

    # --- Configuración de Docling (Método Principal) ---
    print("🚀 Inicializando conversor principal (Docling) con reconocimiento de fórmulas...")
    pipeline_options = PdfPipelineOptions()
    pipeline_options.do_formula_enrichment = True
    converter = DocumentConverter(
        format_options={
            InputFormat.PDF: PdfFormatOption(pipeline_options=pipeline_options)
        }
    )

    try:
        with fitz.open(pdf_path) as doc:
            total_pages = len(doc)
            print(f"📄 Documento '{pdf_path.name}' de {total_pages} páginas. Se procesará en chunks de {chunk_size} páginas.")
    except Exception as e:
        print(f"❌ Error crítico al abrir el PDF: {e}")
        return

    total_time_start = time.time()
    failed_chunks = []

    with open(output_md, "w", encoding="utf-8") as f_out:
        for start_page_idx in range(0, total_pages, chunk_size):
            end_page_idx = min(start_page_idx + chunk_size, total_pages)
            page_start_friendly = start_page_idx + 1
            page_end_friendly = end_page_idx

            chunk_filename = Path(f"/content/temp_chunk_{page_start_friendly}-{page_end_friendly}.pdf")
            
            print(f"\n--- Procesando chunk: Páginas {page_start_friendly} a {page_end_friendly} ---")
            chunk_time_start = time.time()

            # Crear el archivo PDF temporal para el chunk
            try:
                with fitz.open(pdf_path) as original_doc:
                    with fitz.open() as chunk_doc:
                        chunk_doc.insert_pdf(original_doc, from_page=start_page_idx, to_page=end_page_idx - 1)
                        chunk_doc.save(str(chunk_filename))
            except Exception as e:
                print(f"❌ Error al crear el archivo temporal para el chunk. Saltando este chunk. Error: {e}")
                failed_chunks.append(f"{page_start_friendly}-{page_end_friendly}")
                continue # Pasa al siguiente chunk

            # --- LÓGICA DE INTENTO Y FALLBACK ---
            try:
                # 1. Intento Principal: Usar Docling
                print("   -> Intentando con Docling...")
                result = converter.convert(str(chunk_filename))
                md_chunk = result.document.export_to_markdown()
                f_out.write(md_chunk)
                f_out.write("\n\n")
                chunk_time_end = time.time()
                print(f"   ✅ Éxito con Docling en {chunk_time_end - chunk_time_start:.2f} segundos.")

            except Exception as e_docling:
                print(f"   ⚠️ Docling falló: {e_docling}")
                print("   -> Activando fallback con PyMuPDF (extracción de texto simple)...")

                # 2. Fallback: Usar PyMuPDF para extracción de texto
                try:
                    fallback_text = []
                    with fitz.open(chunk_filename) as chunk_doc_fallback:
                        for page in chunk_doc_fallback:
                            fallback_text.append(page.get_text("text"))
                    
                    md_chunk_fallback = "\n".join(fallback_text)
                    
                    # Escribir en el archivo de salida con una advertencia
                    f_out.write(f"\n\n<!-- ADVERTENCIA: El siguiente bloque (páginas {page_start_friendly}-{page_end_friendly}) fue procesado con el método de fallback (PyMuPDF) debido a un error en Docling. El formato puede ser simple. -->\n\n")
                    f_out.write(md_chunk_fallback)
                    f_out.write("\n\n<!-- FIN DEL BLOQUE DE FALLBACK -->\n\n")
                    
                    chunk_time_end = time.time()
                    print(f"   ✅ Éxito con Fallback (PyMuPDF) en {chunk_time_end - chunk_time_start:.2f} segundos.")

                except Exception as e_fallback:
                    # 3. Fallo del Fallback
                    print(f"   ❌ El fallback con PyMuPDF también falló: {e_fallback}")
                    failed_chunks.append(f"{page_start_friendly}-{page_end_friendly}")
                    f_out.write(f"\n\n<!-- ERROR CRÍTICO: No se pudo procesar el chunk de páginas {page_start_friendly}-{page_end_friendly} con ningún método. -->\n\n")

            finally:
                # Limpiar siempre el archivo temporal
                if chunk_filename.exists():
                    os.remove(chunk_filename)

    total_time_end = time.time()
    total_duration_minutes = (total_time_end - total_time_start) / 60
    
    print("\n" + "="*50)
    print(f"🎉 ¡Proceso completo finalizado!")
    print(f"🕒 Tiempo total: {total_duration_minutes:.2f} minutos.")
    print(f"📄 Resultado guardado en: '{output_md}'")
    if failed_chunks:
        print(f"⚠️ Chunks que fallaron completamente: {', '.join(failed_chunks)}")
    print("="*50)

# --- Ejecutar el script robusto ---
process_pdf_with_fallback(
    pdf_path=PDF_FILE, 
    output_md=OUTPUT_MD, 
    chunk_size=CHUNK_SIZE
)